In [93]:
import spacy
import json
from collections import Counter
import pandas as pd
import numpy as np

In [7]:
nlp = spacy.load('C:/Users/mj514/Anaconda3/lib/site-packages/en_core_web_lg/en_core_web_lg-2.0.0')

In [8]:
with open('childrensAuthorContent.json') as infile:
    pages = json.load(infile)

In [27]:
astrid = pages[42].replace('/n', ' ').replace('\n\n\n','').replace('\n','')

In [28]:
astridNLP = nlp(astrid)

In [29]:
nounList = [tok.lemma_ for tok in astridNLP.doc if tok.pos_ == 'NOUN']

In [30]:

count = Counter(nounList)

In [31]:
series = pd.Series(count, name='count')



In [32]:
stopList = list(series.sort_values(ascending=False).head(10).index)
print(stopList)

['–', 'director', 'book', 'child', 'series', 'och', 'year', 'film', 'who', 'tax']


In [33]:
stoppedNouns = [word for word in nounList if word not in stopList]
print(f"before stopping: {len(nounList)}, after: {len(stoppedNouns)}")

before stopping: 548, after: 357


In [34]:
thingsDF  = pd.DataFrame({'thing': stoppedNouns})

In [35]:
thingsDF

,thing
0,ˈasːtrɪd
1,ˈlɪŋːɡreːn
2,writer
3,fiction
4,screenplay
5,fantasy
6,novel
7,daughter
8,house
9,world


In [36]:
people = [ent.merge().text for ent in astridNLP.ents if ent.label_ == 'PERSON']

In [37]:
#https://www.jstor.org/stable/2653656?seq=1#page_scan_tab_contents

str

In [38]:
peopleDF = pd.DataFrame({'name': people})

In [39]:
people[0].__class__

str

In [97]:
numSamples = 10000
sampledPeople = peopleDF.sample(n= numSamples, replace = True).reset_index(drop=True)
sampledThings      = thingsDF.sample(n = numSamples, replace = True,).rename(columns ={'thing':'thing1'}).reset_index(drop=True)
sampledThingsAgain = thingsDF.sample(n = numSamples, replace = True).rename(columns ={'thing':'thing2'}).reset_index(drop=True)
sampledPeople.shape
fullSamples = pd.concat([sampledPeople,sampledThings,sampledThingsAgain], axis = 1)

In [98]:
fullSamples.head(10)

,name,thing1,thing2
0,Lindgren,banknote,theme
1,Pippi Longstocking series,bank,language
2,Samuel August Ericsson,theme,deduction
3,Ingmar Bergman's,center,world
4,Rolf HusbergLuffaren,story,bud
5,Olle HellbomPippi Långstrump,min,author
6,Astrid Lindgren,publishing,memory
7,Rezeption Astrid Lindgrens und,memory,record
8,Göran Persson,death,collection
9,Olle HellbomSkrållan,award,coproduction


In [99]:
fullSamples['first_name'] = fullSamples.name.str[:1].str.lower()
fullSamples = fullSamples[fullSamples['first_name'].str.contains(pat = '[A-Za-z]',regex = True)] #Remove non english characters
fullSamples['first_thing1'] = fullSamples.thing1.str[:1].str.lower()
fullSamples['first_thing2'] = fullSamples.thing2.str[:1].str.lower()

fullSamples['isMagicMatch'] = ((fullSamples.first_name == fullSamples.first_thing1) & (fullSamples.first_name == fullSamples.first_thing2))

In [100]:
np.mean(fullSamples.isMagicMatch)

0.0031465692245229394

In [101]:
fullSamples[fullSamples.isMagicMatch]

,name,thing1,thing2,first_name,first_thing1,first_thing2,isMagicMatch
56,Mio,marriage,memory,m,m,m,True
559,Ronja Rövardotter,rest,run,r,r,r,True
752,Emil,entry,evasion,e,e,e,True
762,Astrid Lindgren,author,author,a,a,a,True
973,Pippi Långstrump,park,prize,p,p,p,True
1600,Emil,effect,election,e,e,e,True
1850,Mio,magazine,memory,m,m,m,True
2218,Stina,site,swede,s,s,s,True
2367,Madicken,museum,museum,m,m,m,True
2550,Pippi Longstocking,punishment,percent,p,p,p,True


In [86]:
len(fullSamples)

9826